# Scala Is... A Better Java

* Simplified Class Definition

```java
// Java:
public class Foo {
  private char c;
  private int n;
  private String s;
  
  public Foo(char c, int n, String s) {
    this.c = c;
    this.n = n;
    this.s = s;
  }
  
  public Foo(char c, int n) {
    this(c, n, "<Nothing>");
  }
  
  public Foo(int n, String s) {
    this('-', n, s);
  }
  
  public Foo(int n) {
    this('-', n, "<Nothing>");
  }
  
  public int getN() {
    return n;
  }
  
  public String getS() {
    return s;
  }
  
  public void setS(String s) {
    this.s = s;
  }
  
  public String toString() {
    return "Foo(" + n + c + s + ")";
  }
}
```

In [1]:
class Foo(c: Char, val n: Int, var s: String = "<Nothing>") {
  def this(n: Int, s: String) = this('-', n, s)
  def this(n: Int) = this('-', n)
  
  override def toString(): String = s"Foo($n$c$s)"
}

val x = new Foo(42)
println(x)
x.s = "Hello"
println(x)

println(x.n)
x.s_=("Goodbye")
println(x.s)

Foo(42-<Nothing>)
Foo(42-Hello)
42
Goodbye


defined class Foo
x: $user.Foo = Foo(42-Goodbye)

* Objects (Singletons) and Companions (Static)

In [2]:
import java.io.File

object Global {
  val VERSION = "1.0"
  var workingDirectory: File = new File("")
  def findFile(name: String): File = new File(workingDirectory, name)
}

class Foo(val n: Int, var s: String) {
  override def toString(): String = s"Foo($n, $s)"
}

object Foo {
  def apply(n: Int, s: String): Foo = new Foo(n, s)
  
  def fromFile(file: File): Foo = {
    import java.util.Scanner
    
    val in = new Scanner(file)
    try {
      val n = in.nextInt()
      val s = in.next()
      new Foo(n, s)
    } finally {
      in.close()
    }
  }
}

Global.workingDirectory = new File("/tmp")
val out = new java.io.PrintWriter(Global.findFile("demo.txt"))
out.println("23 skidoo")
out.close()
Foo.fromFile(Global.findFile("demo.txt"))

import java.io.File
defined object Global
defined class Foo
defined object Foo
out: java.io.PrintWriter = java.io.PrintWriter@56189399
res1_8: $user.Foo = Foo(23, skidoo)

* Multiple, Repeated, Named, and Default Parameters

In [3]:
def thing(args: String*)(sep: String = ",", start: String = "", end: String = ""): String = args.mkString(start, sep, end)

println( thing("foo", "bar", "baz")() )
println( thing("foo", "bar", "baz")(";") )
println( thing("foo", "bar", "baz")(start = "<", end = ">") )
println( thing("foo", "bar", "baz")(start = "<", end = ">", sep = " - ") )

foo,bar,baz
foo;bar;baz
<foo,bar,baz>
<foo - bar - baz>


defined function thing

* By-Name Parameters and Lazy Values

In [4]:
def f(x: => String): Unit = {
  println("f")
  println(x)
  println(x)
}

def g1(x: => String): Unit = {
  println("g1")
  val y = x
}

def g2(x: => String): Unit = {
  println("g2")
  lazy val y = x
}

def h1(x: => String): Unit = {
  println("h1")
  val y = x
  println(y)
  println(y)
}

def h2(x: => String): Unit = {
  println("h2")
  lazy val y = x
  println(y)
  println(y)
}

f{ println("Thinking..."); "Result" }
g1{ println("Thinking..."); "Result" }
g2{ println("Thinking..."); "Result" }
h1{ println("Thinking..."); "Result" }
h2{ println("Thinking..."); "Result" }

f
Thinking...
Result
Thinking...
Result
g1
Thinking...
g2
h1
Thinking...
Result
Result
h2
Thinking...
Result
Result


defined function f
defined function g1
defined function g2
defined function h1
defined function h2

* Traits for Mixin Inheritance

In [5]:
trait Ordered[T] {
  def compare(that: T): Int
  
  def <(that: T): Boolean = (this compare that) < 0
  def >(that: T): Boolean = (this compare that) > 0
  def <=(that: T): Boolean = (this compare that) <= 0
  def >=(that: T): Boolean = (this compare that) >= 0
}

case class Person(first: String, last: String) extends Ordered[Person] {
  def compare(that: Person): Int = {
    val c = last compare that.last
    if (c != 0) c else first compare that.first
  }
}

Person("Brian", "Howard") < Person("Eleanor", "Howard")

defined trait Ordered
defined class Person
res4_2: Boolean = true

In [6]:
trait Renderable {
  def render(width: Int): String = {
    val s = this.toString
    if (s.length <= width) {
      val pad = width - s.length
      "_"*(pad/2) + s + "_"*(pad - pad/2)
    } else {
      s.substring(0, width)
    }
  }
}

case class Person(first: String, last: String) extends Ordered[Person] with Renderable {
  def compare(that: Person): Int = {
    val c = last compare that.last
    if (c != 0) c else first compare that.first
  }
  
  override def toString: String = s"$first $last"
}

Person("Brian", "Howard").render(20)

defined trait Renderable
defined class Person
res5_2: String = "____Brian Howard____"

* Type Parameters (with Declaration-Site Variance)

In [13]:
trait Stack[+T] {
  def top: Option[T]
  def pop: Stack[T]
  def push[U >: T](x: U): Stack[U] = new Stack.StackNode[U](x, this)
}

object Stack {
  def apply[T](args: T*): Stack[T] = args.foldLeft[Stack[T]](EmptyStack)(_ push _)
  
  private object EmptyStack extends Stack[Nothing] {
    def top = None
    def pop = this
  }
  
  private class StackNode[T](val first: T, val rest: Stack[T]) extends Stack[T] {
    def top = Some(first)
    def pop = rest 
  }
}

val s = Stack(1, 2, 3)
val t = s push "Hello"

val x = s.top
val y = t.pop.top

def sum[T](s: Stack[Int]): Int = s.top match {
  case Some(n) => n + sum(s.pop)
  case None => 0
}

sum(s)
// sum(t)

defined trait Stack
defined object Stack
s: $user.Stack[Int] = cmd12$$user$Stack$StackNode@171a76da
t: $user.Stack[Any] = cmd12$$user$Stack$StackNode@4de4a148
x: Option[Int] = Some(3)
y: Option[Any] = Some(3)
defined function sum
res12_7: Int = 6

* Extension Methods with Implicits

In [11]:
implicit class FactInt(val self: Int) /* extends AnyVal */ {
  def ! : BigInt = if (self <= 1) 1 else self * (self - 1).!
}

5!

10!

100!

defined class FactInt
res10_1: BigInt = 120
res10_2: BigInt = 3628800
res10_3: BigInt = 93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000